##  Animating  3D Mobius transformation of a mesh

A Mobius 3d transformation is the composition of two conformal transformations
(the stereographic projection and its inverse) that preserve the angles, but not the lengths, and a rigid motion of a unit sphere in the euclidean space
$\mathbb{R}^4$. As we can see below, a 3d Mobius transformation can rotate and deformate a 3d mesh, in an unexpected way, giving rise to transformed mesh that looks like a creature.

The starting mesh is read from an off file from Faust dataset [http://faust.is.tue.mpg.de/](http://faust.is.tue.mpg.de/):

In [1]:
import numpy as np
import meshio
from mobius3d import *

import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode,  plot
init_notebook_mode(connected=True)

In [2]:
my_lighting =  dict(ambient=0.18,
                    diffuse=1,
                    fresnel=0.1,
                    specular=1, 
                    roughness=0.05,
                    facenormalsepsilon=0)

lightposition = dict(x=200, y=200, z=100)
colorscale = [[0, 'rgb(250,250,250)'], 
              [1, 'rgb(250,250,250)']]


In [7]:
my_mesh =  meshio.read('Data/tr_reg_010.off') 

points = my_mesh.points
points = np.stack((points[:,2], points[:, 0], points[:,1]), axis=0)
x, y, z = points
tri = my_mesh.cells[0].data
I, J, K = tri.T

fig = go.Figure(go.Mesh3d(x=x, y=y, z=z, 
                          i=I, j=J, k=K, 
                          intensity=z, 
                          coloraxis='coloraxis', 
                          lighting=my_lighting,
                          lightposition=lightposition))
title = "3D Mobius transformation with sphere translation  component"
fig.update_layout(title_text = '',
                  title_x= 0.5,
                  font_color='white',
                  coloraxis=dict(colorscale=colorscale,
                                 showscale=False),
                  width=600, height=600, 
                  paper_bgcolor='black'
                 )
fig.update_scenes(xaxis_visible=False,
                  yaxis_visible=False,
                  zaxis_visible=False,
                  aspectmode='data',
                  camera_eye=dict(x=1.80, y=1.80, z=0.5));
fig.add_trace(fig.data[0]) # a trick to draw twice an invisible initial mesh to realize how far is it translated
fig.data[0].update(opacity=0.05);

**Animating Mobius transformations, defined   by sphere admissible translations on the direction $(0,0,0,-1)$**


In [8]:
frames= []
tf = np.linspace(0, 0.95, 45)
t = np.concatenate((tf, tf[:-1][::-1]))
for tt in t:
    xt, yt, zt = Mobius3d_trans(points,  c=[0,0,0,0],  v =[0,0,0, -tt])  
    frames.append(go.Frame(data=[go.Mesh3d(x=xt, y=yt, z=zt, intensity=zt)],
                           traces=[1]))
fig.update(frames=frames);    

In [10]:
fig.update_layout(updatemenus=[dict(type='buttons', 
                                y=0,
                                x=1.15,
                                showactive = False,
                                bgcolor='black',    
                                buttons=[dict(label='Play',
                                              method='animate',
                                              args=[None, 
                                                    dict(frame=dict(duration=100, 
                                                                    redraw=True),
                                                         transition=dict(duration=0),
                                                         fromcurrent=True,
                                                         mode='immediate')]),
                                         dict(label='Pause',
                                              method='animate',
                                              args=[None, 
                                                    dict(frame=dict(duration=0, 
                                                                    redraw=False),
                                                         transition=dict(duration=0),
                                                         fromcurrent=True,
                                                         mode='immediate')])
                                        ])]);
#plot(fig, filename='Mobius-translation', auto_play=False) #uncomment to run it                             

In [12]:
%%html
<img src='Images/Moebius-mesh-transl.gif'>

**Animating Mobius transformations defined by   rotation about `xy` coordinate plane**

Let us re-use the basic settings from the previous figure:

In [13]:
fig1 = go.Figure(fig.data[1], layout=fig.layout)
#fig1.update_layout(title_text="3D Mobius transformation with rotation  as a sphere  mapping")
arg  = fig1.layout.updatemenus[0].buttons[0]['args'][1]
arg['frame'].update(duration=200)
arg['transition'].update(duration=100)

but redefine frames that record mobius transformed meshes, corresponding to rotations $R_2$, with s=0 and t decreasing from 0
to -2*pi

In [14]:
from numpy import pi
frames= []

N = 46
theta= np.linspace(0, 2*pi, N) 
for th in theta:
    Rot = setup_rotation(s=0, t=th)
    xt, yt, zt = Mobius3d_trans(points,  Rot=Rot)  
    frames.append(go.Frame(data=go.Mesh3d(x=xt, y=yt, z=zt, intensity=zt)))
fig1.update(frames=frames); 
#to inspect a particular frame just click pause
#plot(fig1, filename='Mobius-rot-xy.html', auto_play=False)  #UNCOMMENT!

In [15]:
%%html
<img src='Images/Moebius-tr-mesh.gif'>

To inspect the $k^{th}$ frame, $k=0, 1, \ldots, 45$, just run the cell below:

In [16]:
k=29
fig3 = go.Figure(frames[k].data[0], layout=fig1.layout)  #30
fig3.update_traces(i=I, j=J, k=K, coloraxis='coloraxis', 
                          lighting=my_lighting,
                          lightposition=lightposition)

**Animating mesh rotation about an arbitrary plane**

We are generating an arbitrary orthogonal matrix to define the orthogonal decomposition of a rotation, as a Q-component fron the QR-decomposition of an
arbitrary matrix:

In [17]:
def get_orthogonal(A, det_tol=1.e-10):
    #returns the Q matrix from the QR-decomposition of the given matrix, A, and its eigenvals and eigenvect
    if A.ndim !=2 or A.shape[0] != A.shape[1] != 4:
        raise ValueError('A must have the shape (4,4)')
    Q, _ = np.linalg.qr(A)
    return Q

A = np.array([[ 2, -1,  3, -1],
              [-3,  0,  2,  0],
              [ 1,  0,  0,  2],
              [ 0,  1,  3, -1]])
U = get_orthogonal(A)

Define a rotation about the plane generated by the origin, O, and the first two columns in the orthogonal matrix, U, defined above:

In [19]:
frames= []

N = 46
theta= np.linspace(0, 2*pi, N) 
s= np.linspace(0, pi/3, t.shape[0])
for k in range(N):
    Rot = setup_rotation(s=0, t=theta[k], U=U)
    xt, yt, zt = Mobius3d_trans(points,  Rot=Rot)  
    frames.append(go.Frame(data=go.Mesh3d(x=xt, y=yt, z=zt, intensity=zt)))
fig1.update(frames=frames); 
#plot(fig1, filename='Mesh-rot-arbplane.html', auto_play=False) #UNCOMMENT  to run it!

In [20]:
%%html
<img src='Images/Moebius3d-gym.gif'>